# Explore the neighborhood of song recommendations

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import numpy as np
import matplotlib as plt
import importlib

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

## Load and prep data

* Load the full data set
* Vectorize the playlists into sparse vectors
* Extract the vocabulary with tid to allow translation back to track_uri

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

In [ ]:
model, result = mpd.vectorizecol(mpd_all.select("pid", "tracks.track_uri"), "track_uri", "features")

In [ ]:
importlib.reload(mpd)

In [ ]:
vdf = mpd.buildvocabdf(spark, model.vocabulary)

In [ ]:
vdf.show(5)

Demonstrate mapping of tracks in voabulary to human readable names from mpd dataset

In [ ]:
from pyspark.sql.functions import explode
tname=mpd_all.select(explode("tracks").alias("tracks")).select("tracks.track_name", "tracks.track_uri", "tracks.artist_name").distinct()

In [ ]:
vdf.join(tname, tname.track_uri == vdf.term).drop(vdf.term).orderBy("tid").show(5)

## Prepare data for kNN search

* Prep dataset with vector length to elliminate empty playlists from minHash input

In [ ]:
from pyspark.ml.feature import MinHashLSH

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

vectorlength = udf(lambda x: x.numNonzeros(), IntegerType())

In [ ]:
r2=result
r2=r2.withColumn("vlen", vectorlength(r2.features))

In [ ]:
sparsevec = r2.where(r2.vlen > 0)

In [ ]:
sparsevec= sparsevec.rdd.sample(False, .01, 1).toDF()

In [ ]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
model = mh.fit(sparsevec)

In [ ]:
transformA = model.transform(sparsevec)

## Explore results for full playlist match

In [ ]:
testpl=result.select("features").rdd.map(lambda x: x.features).take(1)[0]

In [ ]:
print("Approximately searching dfA for 100 nearest neighbors of the key:")
k100nn = model.approxNearestNeighbors(transformA, testpl, 100)

In [ ]:
mpd.plothist(k100nn, "distCol", 11)

In [ ]:
k100nn.printSchema()

In [ ]:
k100nntracks=k100nn.select(explode("track_uri").alias("track_uri"))

In [ ]:
trackrank = k100nntracks.select("track_uri").groupby("track_uri").count().sort(f.col("count").desc())

In [ ]:
trackrank.printSchema()

In [ ]:
importlib.reload(mpd)

In [ ]:
mpd.scatterplotfreq(trackrank)

## Explore results for a subset of playlist

In [ ]:
testpl

Sparse vectors have an [indices method](http://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.linalg.SparseVector.indices) to get the array elements with values.

In [ ]:
testpl.indices

Getting a subset of the playlist is easy by looking at the indices
They are returned as a numpy array. 
[numpy has a built in to chose a random sample from an array](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)

In [ ]:
rand5npa = np.random.choice(testpl.indices, 5)

Note, the machine learning libararies expect sparse vectors of the new ml.linalg package not the mllib.linalg package.  There is a [conversion method for the old format to return as the new ML format](http://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.linalg.SparseVector.asML).

In [ ]:
from pyspark.ml.linalg import SparseVector 

In [ ]:
np.sort(rand5npa)

In [ ]:
print(testpl.size, np.sort(rand5npa), np.ones(len(rand5npa)))

Use np.sort and np.ones to help create new spase vector for the search

In [ ]:
rand5pl = SparseVector(testpl.size, np.sort(rand5npa), np.ones(len(rand5npa)))

In [ ]:
rand5pl

In [ ]:
k100nn5seed = model.approxNearestNeighbors(transformA, rand5pl, 100)

In [ ]:
k100nn5seed.count()

Note we're only getting 5 candidates so that must mean we [don't have enough candidates in the hash bucket](https://spark.apache.org/docs/2.2.0/ml-features.html#approximate-nearest-neighbor-search).  Wonder if increasing the hash number in the minhash will improve this.

In [ ]:
mpd.plothist(k100nn5seed, "distCol", 11)

## Explore impact of choice of sampled playlist

I would have expected more results not less from a smaller playlist.
I'm assuming that a smaller set of tracks would match a larger collection of playlists.

This could be a luck of the draw for the random selection. Repeating the random choice selection from the playlist now gives me 11 results.

So this is challenging. How to ensure I get enough results from the search

I assume what is happening is that if there are tracks that are less popular in the search then I'm going to be more of an edge case playlist.

Yeah, and if i take just the top-5 most popular songs in the playlist then i get my 100 results as expected.

Randomly choose tracks from playlist. Leads to small result set for near neighbors

In [ ]:
rand5npa = np.random.choice(testpl.indices, 5)

Choose 5 most popular songs from playlist. Leads to high number of near neighbors.

In [ ]:
rand5npa = np.sort(testpl.indices)[0:5]

Choose the first 5 songs from the playlist. This is the continuation model.  You hear 5 songs and now recommend the rest.

Note, in the current construction of testpl we have lost the orignal playlist order since we are deriving this from the sparse vector.

Really need to go back and redefine testpl as coming from the original playlist

In [ ]:
rand5npa = testpl.indices[0:5]

In [ ]:
rand5pl = SparseVector(testpl.size, np.sort(rand5npa), np.ones(len(rand5npa)))

In [ ]:
rand5pl

In [ ]:
mh10 = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=10)

In [ ]:
model10 = mh10.fit(sparsevec)

In [ ]:
transformA10 = model10.transform(sparsevec)

In [ ]:
k100nn5seed10 = model.approxNearestNeighbors(transformA10, rand5pl, 100)

In [ ]:
k100nn5seed10.count()

In [ ]:
mpd.plothist(k100nn5seed10, "distCol", 11)

## Explore artists as clustering

In [ ]:
amodel, aresult = mpd.vectorizecol(mpd_all.select("pid", "tracks.artist_uri"), "artist_uri", "features")

In [ ]:
avdf = mpd.buildvocabdf(spark, amodel.vocabulary)

In [ ]:
avdf.show(5)

In [ ]:
aname=mpd_all.select(explode("tracks").alias("tracks")).select("tracks.artist_uri", "tracks.artist_name").distinct()

In [ ]:
avdf.join(aname, aname.artist_uri == avdf.term).drop(avdf.term).orderBy("tid").show(5)

In [ ]:
ar2=aresult
ar2=ar2.withColumn("vlen", vectorlength(ar2.features))

In [ ]:
ar2.count()

In [ ]:
asparsevec = ar2.where(ar2.vlen > 0)

In [ ]:
asparsevec.count()

In [ ]:
asparsevec= asparsevec.rdd.sample(False, .01, 1).toDF()

In [ ]:
amh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
amhmodel = amh.fit(asparsevec)

In [ ]:
amhtransform = amhmodel.transform(asparsevec)

In [ ]:
atestpl=aresult.select("features").rdd.map(lambda x: x.features).take(1)[0]

In [ ]:
atestpl

In [ ]:
print("Approximately searching dfA for 100 nearest neighbors of the artist:")
ak100nn = model.approxNearestNeighbors(amhtransform, atestpl, 100)

In [ ]:
mpd.plothist(ak100nn, "distCol", 11)

In [ ]:
k100nnartists=ak100nn.select(explode("artist_uri").alias("artist_uri"))

In [ ]:
artistrank = k100nnartists.select("artist_uri").groupby("artist_uri").count().sort(f.col("count").desc())

In [ ]:
mpd.scatterplotfreq(artistrank)

### Explore subset of playlist match with artists

In [ ]:
atestpl

In [ ]:
arand5npa = np.random.choice(atestpl.indices, 5, replace=False)

In [ ]:
arand5npa

In [ ]:
arand5pl = SparseVector(atestpl.size, np.sort(arand5npa), np.ones(len(arand5npa)))

In [ ]:
ak100nn5seed = model.approxNearestNeighbors(amhtransform, arand5pl, 100)

In [ ]:
ak100nn5seed.count()

In [ ]:
ak100nn5seed.printSchema()

In [ ]:
ak100nn5seed.orderBy("pid").show(5)

So the playlist name matches are less perfect than with the full playlist search, at least based on superficial name matching.

Get the sense that searching for knn with the full sample playlist by tracks got many similarly named playlists.

In [ ]:
ak100nn5seed.join(mpd_all.select("pid", "name"), mpd_all.pid == ak100nn5seed.pid).drop(mpd_all.pid).select("pid", "name").show(5)

## Explore knn with full vocab and sample playist

See how it works to search against a larger playlist database.  Do small samples of just 5 songs return larger neighborhood.

In [ ]:
importlib.reload(mpd)

In [ ]:
model, result = mpd.vectorizecol(mpd_all.select("pid", "tracks.track_uri"), "track_uri", "features", size=1188873 )

In [ ]:
mhall = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
modelall = mhall.fit(sparsevec)

In [ ]:
transformall = modelall.transform(sparsevec)

In [ ]:
k100nn5seedall = modelall.approxNearestNeighbors(transformall, rand5pl, 100)

In [ ]:
k100nn5seedall.count()

With the full playlist collection of all songs that have more than one occurance the neighborhood results are much better. The search performance doesn't seem to have changed much either. 

In [ ]:
mpd.plothist(k100nn5seedall, "distCol", 11)

In [ ]:
k100nn5seedall.join(mpd_all.select("pid", "name"), mpd_all.pid == k100nn5seedall.pid).drop(mpd_all.pid).select("pid", "name").show(5)

### Explore quality of music recommendation

Get top 500 recommended songs from 100 recommended playlists based on 5 tracks.

In [ ]:
k100nn5seedall.printSchema()

In [ ]:
k100nn5seedall.select(explode(k100nn5seedall.track_uri).alias("track_uri")).groupBy("track_uri").count().sort(f.col("count").desc()).show(5)

In [ ]:
trackrank=k100nn5seedall.select(explode(k100nn5seedall.track_uri).alias("track_uri")).groupBy("track_uri").count().sort(f.col("count").desc())

In [ ]:
Y=trackrank.select("count").toPandas()

In [ ]:
X=pd.DataFrame({'X': range(1,Y.size+1,1)})

In [ ]:
plt.pyplot.scatter(X,Y)

### Get track info from knn recommended neighbors

In [ ]:
k100nn5seedall = k100nn5seedall.withColumnRenamed("pid", "recpid")

In [ ]:
pln=k100nn5seedall.join(mpd_all, mpd_all.pid == k100nn5seedall.recpid)

In [ ]:
pln.printSchema()

In [ ]:
pln.select("name", "modified_at", "num_edits", "num_followers","num_tracks").show()

In [ ]:
pDF=pln.select("pid", explode("tracks").alias("track")).select("track.*")

In [ ]:
pDF.select("track_name").show()

In [ ]:
pDF.count()

### Get knn playlist and global ranking

In [ ]:
tvdf = mpd.buildvocabdf(spark, model.vocabulary)

In [ ]:
tvdf.printSchema()

In [ ]:
grank=trackrank.join(tvdf, trackrank.track_uri == tvdf.term)

In [ ]:
grank.printSchema()

In [ ]:
grank.orderBy(f.desc("count"), f.desc("tid")).show()

In [ ]:
recommendrank=grank.orderBy(f.desc("count"), f.desc("tid")).select("track_uri", "count", "tid")

In [ ]:
rand5pl

In [ ]:
testpl

[Convert a numpy array to a dataframe via a pandas dataframe](https://stackoverflow.com/a/46818308)

In [ ]:
rand5pl.indices

In [ ]:
type(rand5pl.indices)

In [ ]:
from pyspark.sql.types import StringType

In [ ]:
r5pldf = spark.createDataFrame(rand5pl.indices, StringType())

In [ ]:
r5pldf = spark.createDataFrame(pd.DataFrame(rand5pl.indices), ["idx"])

In [ ]:
r5pldf.show()

In [ ]:
recommendrank.join(r5pldf, recommendrank.tid == r5pldf.idx).show()

In [ ]:
recommendrank.join(r5pldf, recommendrank.tid != r5pldf.idx).show()

Want to see the intersection of the recommended tracks and the tracks that were held out of the search playlist.

This is hard to express in sql because a not-equal between columns doesn't reflect the semantics of not in.  That just elliminates the search playlist tracks from the recommended result.

It might be quick to eliminate the search tracks from the full playlist with a dot product?  No cause that sums them up.  What i really want is a mask.  Feel this should be easier with sql.

There is a [not in clause with negation ~](https://stackoverflow.com/a/40607572)

In [ ]:
rand5pl.indices

In [ ]:
recommendrank.where(~recommendrank.tid.isin(rand5pl.indices.tolist())).show(5)